# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с Selenium: пример работы  с ВКонтакте

### Часть 1: залогиниваемся во ВКонтакте

Импортируем необходимые компоненты Selenium:

In [ ]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Импортируем функцию для запроса пароля, который скрывает вводимые с клавиатуры символы:

In [ ]:
from getpass import getpass

Открываем браузер – начинаем новую сессию работы с Selenium:

In [ ]:
br = wd.Chrome(executable_path='/Users/allat/Desktop/chromedriver')

Переходим в браузере на главную страницу ВКонтакте:

In [ ]:
br.get("https://vk.com/")

Находим поле для логина:

In [ ]:
login = br.find_element(By.ID, "index_email")

Отправляем туда свой логин:

In [ ]:
login.send_keys("allatambov@mail.ru")

Чтобы не искать на странице кнопку *Войти* (не очень удобно), просто имитируем нажатие кнопки *Enter*:

In [ ]:
login.send_keys(Keys.ENTER)

Находим поле для ввода пароля – тут надо постараться, потребуется просмотр кода страницы через интсрументы разработчика:

In [ ]:
password = br.find_element(By.NAME, "password")

Запрашиваем пароль с клавиатуры у самих себя:

In [ ]:
my_password = getpass()

Отправляем пароль в соответствующее поле и снова имитируем нажатие *Enter*:

In [ ]:
password.send_keys(my_password)

In [ ]:
password.send_keys(Keys.RETURN)

Готово! Можно искать информацию!

### Часть 2: ищем пользователей

Перейдём на страницу поиска друзей:

In [ ]:
br.get("https://vk.com/friends")

Найдём на этой странице ссылку с текстом *Поиск друзей*, чтобы перейти к странице поиска пользователей по заданным критериям, и кликнем на неё:

In [ ]:
search = br.find_element(By.LINK_TEXT, "Поиск друзей")
search.click() 

Чтобы формировать критерии для поиска, нам нужно открыть меню с фильтрами – развернуть меню *Параметры поиска*. Изучив исходный код страницы, видим, что параметры поиска можно найти по id:

In [ ]:
pars = br.find_element(By.ID, "friends_filters_block")
pars.click()

Для начала выберем город. В исходном коде страницы поле для ввода города имеет тэг `<input>` и id, равный `cCity`:

In [ ]:
city = br.find_element(By.ID, "cCity")

Объект `city` – это объект типа `webelement.WebElement`, то есть элемент страницы, с которым умеет работать модуль `webdriver` из `selenium`. По такому объекту тоже можно выполнять поиск с помощью метода `find_element`. 

В нашем случае внутри этого элемента нужно найти другой, с тэгом `<input>`, потому что нас интересует поле для ввода значения. Если этим шагом пренебречь и попытаться ввести название страны прямо в `city`, мы получим ошибку вида `element not interactable`, потому что сам по себе раздел со страной никакого взаимодействия с пользователем не предполагает, его нельзя редактировать, на него нельзя кликать и прочее.

Поэтому найдём внутри `city` поле для ввода значения по тэгу:

In [ ]:
city_inp = city.find_element(By.TAG_NAME, "input")
city_inp.send_keys("Москва") 

Отлично! Значение выбрано. Но есть проблема – оно «повисло» в воздухе, опция с выбором страны отображается как выбранная в выпадающем меню, но в самом поле выбор не зафиксирован. Чтобы подтвердить выбор, нужно нажать на клавишу *Enter*:

In [ ]:
br.implicitly_wait(3)  # задержка
city_inp.send_keys(Keys.RETURN)

А вот с полом всё поинтереснее: найти поле для ввода пола просто, а вот значения нужно выбирать, нажимая на радиокнопки (*radiobuttons*). Сначала найдём поле для выбора пола:

In [ ]:
sex = br.find_element(By.ID, "cSex")

А теперь – все опции внутри (согласно исходному коду, они имеют тэг `<div>`):

In [ ]:
values = sex.find_elements(By.TAG_NAME, "div")
values

Обратите внимание: здесь метод `find_elements()`, не `find_element()`, потому что результатов ожидается несколько. Методы вида `find_element()` возвращают только первое совпадение на странице, методы вида `find_elements()` – все совпадения на странице (можно провести аналогию с `find` и `find_all` в `BeautifulSoup`).

Выбираем мужской пол – это второй элемент списка – и кликаем на него:

In [ ]:
values[1].click()

На этом закончим работу с фильтрами и перейдём к результатам. На этом этапе возможности `selenium` нам пока не понадобятся, нам нужно только запросить исходный код страницы, которая сейчас открыта в окне браузера, управляемом из Python, и продолжить работу с HTML с помощью BeautfulSoup.

In [ ]:
html = br.page_source

### Часть 3: обработка результатов

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(html)

### Задача 1

Создайте список `divs` с фрагментами html-кода, каждый из которых соответствует одной «карточке» пользователя с именем и ссылкой на профиль.

In [ ]:
### YOUR CODE HERE ###

### Задача 2

Напишите функцию `get_person()` для извлечения имени и ссылки на профиль пользователя (ссылки должны быть полными) и примените её ко всем пользователям на странице. Сохраните результаты в датафрейм `people`.

In [ ]:
### YOUR CODE HERE ###